# Analyze superstore Data in Jupyter

In [1]:
import pandas as pd
import json

#load data train.csv from ../data/train.csv
df = pd.read_csv("../data/train.csv")
# Display basic info
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


## Analyze Cardinality

In [2]:
# Check unique values and cardinality
print("Unique Values and Cardinality:")
for col in ["Customer ID", "Region", "Product ID", "Order ID"]:
    unique_count = df[col].nunique()
    total_count = len(df)
    print(f"{col}: {unique_count} unique values ({unique_count/total_count*100:.2f}% cardinality)")

Unique Values and Cardinality:
Customer ID: 793 unique values (8.09% cardinality)
Region: 4 unique values (0.04% cardinality)
Product ID: 1861 unique values (18.99% cardinality)
Order ID: 4922 unique values (50.22% cardinality)


- `CustomerID`: High cardinality, good for sharding.
- `Region`: Low cardinality, risks hotspots.
- `ProductID`: Medium cardinality, viable alternative.
- `OrderID`: Too high cardinality, inefficient for sharding.

## Analyze Distribution

In [4]:
# Distribution of key fields
print("Distribution by Region:")
print(df["Region"].value_counts())
print("\nDistribution by Customer ID (Top 5):")
print(df["Customer ID"].value_counts().head())
print("\nDistribution by Product ID (Top 5):")
print(df["Product ID"].value_counts().head())

Distribution by Region:
Region
West       3140
East       2785
Central    2277
South      1598
Name: count, dtype: int64

Distribution by Customer ID (Top 5):
Customer ID
WB-21850    35
MA-17560    34
PP-18955    34
JL-15835    33
JD-15895    32
Name: count, dtype: int64

Distribution by Product ID (Top 5):
Product ID
OFF-PA-10001970    19
TEC-AC-10003832    18
FUR-FU-10004270    16
TEC-AC-10003628    15
FUR-CH-10002647    15
Name: count, dtype: int64


- `Region`: Skewed (e.g., West dominates), poor shard key.
- `CustomerID`: Fairly even distribution, good for sharding.
- `ProductID`: Slightly skewed, but usable.

## Analyze Query Patterns

In [6]:
# Sales by Region
print("Sales by Region:")
print(df.groupby("Region")["Sales"].sum().sort_values(ascending=False))

# Orders per Customer
print("\nOrders per Customer (Top 5):")
print(df.groupby("Customer ID")["Order ID"].count().sort_values(ascending=False).head())

# Orders per product
print("\nOrders per product (Top 5):")
print(df.groupby("Product ID")["Order ID"].count().sort_values(ascending=False).head())

Sales by Region:
Region
West       710219.6845
East       669518.7260
Central    492646.9132
South      389151.4590
Name: Sales, dtype: float64

Orders per Customer (Top 5):
Customer ID
WB-21850    35
PP-18955    34
MA-17560    34
JL-15835    33
SV-20365    32
Name: Order ID, dtype: int64

Orders per product (Top 5):
Product ID
OFF-PA-10001970    19
TEC-AC-10003832    18
FUR-FU-10004270    16
TEC-AC-10003628    15
FUR-CH-10002647    15
Name: Order ID, dtype: int64


- `Sales by Region`: Common query, but sharding by Region risks uneven shards.
- `Orders per Customer` : Frequent customer-specific queries (like who is our best customer analytics) make Customer ID a good choice for data locality.
- `Orders per product`: Seems a close second for a shard choice. What is our most bought product?

## Decide on Shard Key
- Based on the Jupyter analysis:
- Recommended Shard Key: CustomerID (hashed)
- Why:
    - High Cardinality: ~8% (e.g., ~800 customers in 9800 orders) ensures even distribution across shards.
    - Even Distribution: value_counts() shows no single customer dominates, avoiding hotspots.
    - Query Efficiency: Customer-specific queries (e.g., orders per customer) are common in Guidewire-like scenarios (policyholder analytics), and sharding by CustomerID localizes data.
    - Hashed Index: Randomizes distribution to prevent sequential CustomerID clumping.